In [3]:
import pandas as pd
from pathlib import Path

month = 1
year = 2024
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.csv"

ts_data = pd.read_csv(path)
ts_data.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_17968\2649026679.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ts_data = pd.read_csv(path)


,pickup_hour,pickup_location_id,rides
0,2024-01-01 00:00:00,2733.03,0
1,2024-01-01 01:00:00,2733.03,0
2,2024-01-01 02:00:00,2733.03,0
3,2024-01-01 03:00:00,2733.03,0
4,2024-01-01 04:00:00,2733.03,0


In [5]:
top_locations = (
    ts_data.groupby("pickup_location_id")["rides"]
    .sum()
    .sort_values(ascending=False)
    .head(3)
)
top_locations

pickup_location_id
6140.05    6925
6822.09    6554
5779.11    5873
Name: rides, dtype: int64

In [6]:
ts_data[ts_data["pickup_location_id"]==6140.05].head(36)

,pickup_hour,pickup_location_id,rides
773760,2024-01-01 00:00:00,6140.05,0
773761,2024-01-01 01:00:00,6140.05,1
773762,2024-01-01 02:00:00,6140.05,1
773763,2024-01-01 03:00:00,6140.05,0
773764,2024-01-01 04:00:00,6140.05,0
773765,2024-01-01 05:00:00,6140.05,0
773766,2024-01-01 06:00:00,6140.05,0
773767,2024-01-01 07:00:00,6140.05,0
773768,2024-01-01 08:00:00,6140.05,0
773769,2024-01-01 09:00:00,6140.05,0


In [7]:
import pandas as pd
import numpy as np

def transform_time_series_to_tabular(df, location_id, feature_col="rides", window_size=12, step_size=1):
    """
    Transforms time series data for a given location ID into a tabular format.
    The first `window_size` rows are used as features, and the next row is the target.
    The process slides down by `step_size` rows at a time to create the next set of features and target.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing time series data.
        location_id (int): The location ID to filter the data for.
        feature_col (str): The column name containing the values to use as features and target (default is "rides").
        window_size (int): The number of rows to use as features (default is 12).
        step_size (int): The number of rows to slide the window by (default is 1).

    Returns:
        pd.DataFrame: A transformed DataFrame where the first `window_size` columns are features
                      and the last column is the target.
    """
    # Filter the data for the given location ID
    location_data = df[df["pickup_location_id"] == location_id].reset_index(drop=True)

    # Extract the feature column as a NumPy array
    values = location_data[feature_col].values

    # Ensure there are enough rows to create at least one window
    if len(values) <= window_size:
        raise ValueError("Not enough data to create even one window of features and target.")

    # Create the tabular data using a sliding window approach
    rows = []
    for i in range(0, len(values) - window_size, step_size):
        # The first `window_size` values are features, and the next value is the target
        features = values[i:i + window_size] 
        target = values[i + window_size] 
        rows.append(np.append(features, target))

    # Convert the list of rows into a DataFrame
    column_names = [f"feature_{i+1}" for i in range(window_size)] + ["target"]
    transformed_df = pd.DataFrame(rows, columns=column_names)

    return transformed_df

In [8]:
# hours = 3
# days = 7
# window_size = hours * days
features_targets = transform_time_series_to_tabular(ts_data,6140.05, "rides", 24, 1)

In [9]:
features_targets

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,target
0,0,1,1,0,0,0,0,0,0,0,...,1,0,2,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,2,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,2,...,2,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,2,4,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,2,4,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,3,3,2,1,0,1,0,0,0,0,...,3,7,1,3,4,4,7,14,6,9
716,3,2,1,0,1,0,0,0,0,1,...,7,1,3,4,4,7,14,6,9,1
717,2,1,0,1,0,0,0,0,1,0,...,1,3,4,4,7,14,6,9,1,2
718,1,0,1,0,0,0,0,1,0,2,...,3,4,4,7,14,6,9,1,2,0
